Group Members: Vakkalanka V S S Dilip Raju (vvakka2), Lakshmi Kamala Kotha(lkotha2)
Installations:
    installed NLTK and stopwords
    !pip install nltk
    nltk.download()[installed stop words]

Below Cell will import all the neccessary libararies and will print a success message if the import is successful
Removed punctuations, http(s),www
Provided some regular expressions for processing the tweet

In [1]:
import os
import sys
import string
import csv,re,io
import nltk
import pyspark
from pyspark.sql import SQLContext
try:
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    from pyspark import SparkContext
    from pyspark import SparkConf
    from pyspark.mllib.feature import Normalizer
    from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
    from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
    from pyspark.mllib.linalg import Vectors
    from pyspark.mllib.regression import LabeledPoint
    from pyspark.mllib.feature import HashingTF
    from pyspark.mllib.feature import IDF
    from pyspark.mllib.feature import StandardScaler
    from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel,LogisticRegressionWithSGD
    from pyspark.mllib.evaluation import BinaryClassificationMetrics,MulticlassMetrics
    from pyspark.mllib.util import MLUtils 
    from pyspark.ml.feature import NGram
    from pyspark.sql import SQLContext
    print("Successfully imported Spark Modules")
except ImportError as e:
    print("Error importing Spark Modules", e)
    sys.exit(1)

punc = string.punctuation.replace('_', '')

punc_regex = re.compile('[%s]' % re.escape(punc))
url_regex = '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'
nonalpha1 = re.compile('[0-9]+[a-z]+')
nonalpha2 = re.compile('\s[0-9]+\s')
one_two = '\s+\w\w?\s+'

Successfully imported Spark Modules


#### Pre Processing:

Below function will perform all the neccessary processing steps on the tweet and parse_line will return the tweet with its output variable.

In [2]:
def remove_punc(s):
    temp = re.sub( url_regex, 'url', s).strip()  # Remove URLs from the text
    temp = re.sub('^@([A-Za-z0-9_]+)', 'atuser', temp).strip()  # Replace user names with AT_USER
    temp = nonalpha1.sub(' ',temp)
    temp = nonalpha2.sub(' ',temp)
    temp = re.sub(r'(.)\1{2,}',r'\1', temp)
    temp = punc_regex.sub(' ', temp).strip()  # Replace punctuation with out '_'
    temp = re.sub(one_two, ' ', temp)
    return re.sub( '\s+', ' ', temp).strip()


def parse_line(line):
    columns = csv.reader(io.StringIO(line))
    k = columns.__next__()
    polarity = int(k[0].strip('"'))
    tweet = k[5].strip().lower()
    tweet = remove_punc(tweet)
    return tweet, polarity

### Stemming and Stopwords removal

Below cell will perform stemming and removes stop words from nltk stopwords list.

In [3]:
STEMMER = PorterStemmer()

STOPWORDS = set(stopwords.words('english'))

def tokenizer(line):
    tweet_words = line.strip().split(" ")
    tweet_nostopwords = [word for word in tweet_words if not word in STOPWORDS]
    tweet_stemmed = [STEMMER.stem(word) for word in tweet_nostopwords]
    return [word for word in tweet_stemmed if word]

### Data Retrieval 

Below cell will retreive test and train data. It will perform the above preprocessing steps and tokenizes the data.

In [4]:
if __name__ == '__main__':
    conf = (SparkConf().setAppName("TwitterSA").set("spark.executor.memory", "6g"))
    #sc = SparkContext(conf=conf)
    sc = pyspark.SparkContext('local[*]')
    sqlContext = SQLContext(sc)
    input_train = "train.csv"
    input_test = "test.csv"

    raw_train = sc.textFile(input_train)
    print('## Parsing Train Data...')
    train_data = raw_train.map(parse_line)
    print('## Tokenizing Train Data...')
    train_data = train_data.map(lambda x: (tokenizer(x[0]),x[1]))
    train_data.cache()
    # print(train_data.take(3))

    raw_test = sc.textFile(input_test)
    print('## Parsing Test Data...')
    test_data = raw_test.map(parse_line)
    print('## Tokenizing Test Data...')
    test_data = test_data.map(lambda x: (tokenizer(x[0]),x[1]))
    test_data.cache()
    # print(test_data.take(3))

## Parsing Train Data...
## Tokenizing Train Data...
## Parsing Test Data...
## Tokenizing Test Data...


#### Bigram feature extraction

In [5]:
def biGramfunc(tweet):
    wordDataFrame = sqlContext.createDataFrame(tweet, ["words","polarity"])
    ngram = NGram(inputCol="words", outputCol="ngrams")
    ngramDataFrame = ngram.transform(wordDataFrame)
    #for ngrams_label in ngramDataFrame.take(3):
    #    print(ngrams_label)
    ngrams_rdd = ngramDataFrame.select("polarity","ngrams").rdd.map(lambda x:[x[1],x[0]])
    #print(ngrams_rdd.take(3))
    return ngrams_rdd

#### Naive Bayes Classifier Unigram

Used 50000 as the number of features. (We need to modify it to 4000 for running decision trees). We built labeled training and test data which contains both the class label and tweet(which is transformed into TF vector using hashtingTF))

Trained the naive bayes using labeled train data and calculated both the train and test accuracy.

Found different metrics using multiclass metrics and found the area of the roc curve using BinaryClassificationMetrics

In [6]:
    hashingTF = HashingTF(50000)
    ### Start of Naive bayes

    print('## Building model on Train Data...')
    
    labeled_training_data = train_data.map(lambda x: LabeledPoint(x[1],hashingTF.transform(x[0])))
    labeled_training_data.persist()
    print(labeled_training_data.take(2))
    labeled_testing_data = test_data.map(lambda x: LabeledPoint(x[1],hashingTF.transform(x[0])))
    labeled_testing_data.persist()
    
    NBmodel = NaiveBayes.train(labeled_training_data,lambda_=1.0)
    
    
    NB_P_O_train = labeled_training_data.map(lambda p: (NBmodel.predict(p.features), p.label))
    accuracy = 1.0 * NB_P_O_train.filter(lambda x: x[0] == x[1]).count() / labeled_training_data.count()
    print("Training Data",accuracy)
    #
    NB_P_O_test = labeled_testing_data.map(lambda p:(float(NBmodel.predict(p.features)), p.label))
    accuracy = 1.0 * NB_P_O_test.filter(lambda x: x[0] == x[1]).count() / labeled_testing_data.count()
    print("Testing Data",accuracy)
    
    # Instantiate metrics object
    metrics = MulticlassMetrics(NB_P_O_test)
    
    # Overall statistics
    precision = metrics.precision()
    recall = metrics.recall()
    f1Score = metrics.fMeasure()
    confMatrix=metrics.confusionMatrix().toArray()
    print(confMatrix)
    print("Summary Stats")
    print("Precision = %s" % precision)
    print("Recall = %s" % recall)
    print("F1 Score = %s" % f1Score)
    # Statistics by class
    labels = [0.0,1.0]
    for label in sorted(labels):
        print("Class %s precision = %s" % (label, metrics.precision(label)))
        print("Class %s recall = %s" % (label, metrics.recall(label)))
        print("Class %s F1 Measure = %s\n\n" % (label, metrics.fMeasure(label, beta=1.0)))
    metrics=BinaryClassificationMetrics(NB_P_O_test)
    print("Area under ROC =%s"% metrics.areaUnderROC)

    ### End of Naive bayes

## Building model on Train Data...
[LabeledPoint(0.0, (50000,[703,2553,8539,11402,21076,21596,24604,37581,41977],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), LabeledPoint(0.0, (50000,[2553,8539,20451,24323,25651],[1.0,1.0,1.0,1.0,1.0]))]
Training Data 0.8041375
Testing Data 0.8105849582172702
[[ 143.   34.]
 [  34.  148.]]
Summary Stats
Precision = 0.8105849582172702
Recall = 0.8105849582172702
F1 Score = 0.8105849582172702
Class 0.0 precision = 0.807909604519774
Class 0.0 recall = 0.807909604519774
Class 0.0 F1 Measure = 0.8079096045197741


Class 1.0 precision = 0.8131868131868132
Class 1.0 recall = 0.8131868131868132
Class 1.0 F1 Measure = 0.8131868131868132


Area under ROC =0.8105482088532936


### Bigram implementation of naive bayes

In [10]:
    hashingTF = HashingTF(50000)
    ### Start of Naive bayes

    print('## Building model on Train Data...')
    
    labeled_training_data = biGramfunc(train_data).map(lambda x: LabeledPoint(x[1],hashingTF.transform(x[0])))
    labeled_training_data.persist()
    print(labeled_training_data.take(2))
    labeled_testing_data =biGramfunc(test_data).map(lambda x: LabeledPoint(x[1],hashingTF.transform(x[0])))
    labeled_testing_data.persist()
    
    NBmodel = NaiveBayes.train(labeled_training_data,lambda_=2.0)
    
    
    NB_P_O_train = labeled_training_data.map(lambda p: (NBmodel.predict(p.features), p.label))
    accuracy = 1.0 * NB_P_O_train.filter(lambda x: x[0] == x[1]).count() / labeled_training_data.count()
    print("Training Data",accuracy)
    #
    NB_P_O_test = labeled_testing_data.map(lambda p:(float(NBmodel.predict(p.features)), p.label))
    accuracy = 1.0 * NB_P_O_test.filter(lambda x: x[0] == x[1]).count() / labeled_testing_data.count()
    print("Testing Data",accuracy)
    
    # Instantiate metrics object
    metrics = MulticlassMetrics(NB_P_O_test)
    
    # Overall statistics
    precision = metrics.precision()
    recall = metrics.recall()
    f1Score = metrics.fMeasure()
    confMatrix=metrics.confusionMatrix().toArray()
    print(confMatrix)
    print("Summary Stats")
    print("Precision = %s" % precision)
    print("Recall = %s" % recall)
    print("F1 Score = %s" % f1Score)
    # Statistics by class
    labels = [0.0,1.0]
    for label in sorted(labels):
        print("Class %s precision = %s" % (label, metrics.precision(label)))
        print("Class %s recall = %s" % (label, metrics.recall(label)))
        print("Class %s F1 Measure = %s\n\n" % (label, metrics.fMeasure(label, beta=1.0)))
    metrics=BinaryClassificationMetrics(NB_P_O_test)
    print("Area under ROC =%s"% metrics.areaUnderROC)

    ### End of Naive bayes

## Building model on Train Data...
[LabeledPoint(0.0, (50000,[1657,4556,10298,10973,31939,35220,46417,49002],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])), LabeledPoint(0.0, (50000,[8311,11502,28608,47002],[1.0,1.0,1.0,1.0]))]
Training Data 0.8152375
Testing Data 0.5793871866295265
[[  92.   85.]
 [  66.  116.]]
Summary Stats
Precision = 0.5793871866295265
Recall = 0.5793871866295265
F1 Score = 0.5793871866295265
Class 0.0 precision = 0.5822784810126582
Class 0.0 recall = 0.519774011299435
Class 0.0 F1 Measure = 0.5492537313432835


Class 1.0 precision = 0.5771144278606966
Class 1.0 recall = 0.6373626373626373
Class 1.0 F1 Measure = 0.6057441253263708


Area under ROC =0.5785683243310362


### Logistic Regression

Performed TF IDF on both training and test data and supplied it to Logistic Regression Stochastic Gradient descent.

In [11]:
    print('## Calculating TF*IDF for Train Data...')
    train_tf = train_data.map(lambda x: hashingTF.transform(x[0]))
    train_tf.cache()
    idf = IDF().fit(train_tf)
    train_tfidf = idf.transform(train_tf)
    #train_tf.unpersist()
    print('## Creating Labeled points for Train Data...')
    tfidf_index = train_tfidf.zipWithIndex()
    training_index = train_data.zipWithIndex()
    index_training = training_index.map(lambda line: (line[1], line[0][1]))#label,
    index_tfidf = tfidf_index.map(lambda l: (l[1], l[0]))#index,vector
    joined_tfidf_training = index_training.join(index_tfidf).map(lambda x: x[1])#label,vector
    # print(training_labeled.take(2))
    labeled_training_data = joined_tfidf_training.map(lambda k: LabeledPoint(k[0], k[1]))
    # print(labeled_training_data.take(2))
    labeled_training_data.cache()
    #train_data.unpersist()



    print('## Calculating TF*IDF for Test Data...')
    test_tf = test_data.map(lambda x: hashingTF.transform(x[0]))
    test_tf.cache()
    #idf = IDF().fit(test_tf)
    test_tfidf = idf.transform(test_tf)
    test_tf.unpersist()
    print('## Creating Labeled points for Test Data...')
    tfidf_index = test_tfidf.zipWithIndex()
    testing_index = test_data.zipWithIndex()
    index_testing = testing_index.map(lambda line: (line[1], line[0][1]))
    index_tfidf = tfidf_index.map(lambda l: (l[1], l[0]))
    joined_tfidf_testing = index_testing.join(index_tfidf).map(lambda x: x[1])
    # print(testing_labeled.take(2))
    labeled_testing_data = joined_tfidf_testing.map(lambda k: LabeledPoint(k[0], k[1]))
    # print(labeled_testing_data.take(2))
    labeled_testing_data.cache()
    #test_data.unpersist()

    LRmodel = LogisticRegressionWithSGD.train(labeled_training_data)
    #
    LRmodel.setThreshold(0.5)
    LR_P_O_train = labeled_training_data.map(lambda p: (LRmodel.predict(p.features), p.label))
    accuracy = 1.0 * LR_P_O_train.filter(lambda x: x[1] == x[0]).count() / labeled_training_data.count()
    print("Training Data",accuracy)
    #
    LR_P_O_test = labeled_testing_data.map(lambda p: (float(LRmodel.predict(p.features)), p.label))
    accuracy = 1.0 * LR_P_O_test.filter(lambda x: x[1] == x[0]).count() / labeled_testing_data.count()
    print("testing Data",accuracy)
    
    # Instantiate metrics object
    metrics = MulticlassMetrics(LR_P_O_test)
    
    # Overall statistics
    precision = metrics.precision()
    recall = metrics.recall()
    f1Score = metrics.fMeasure()
    confMatrix=metrics.confusionMatrix().toArray()
    print(confMatrix)
    print("Summary Stats")
    print("Precision = %s" % precision)
    print("Recall = %s" % recall)
    print("F1 Score = %s" % f1Score)
    # Statistics by class
    labels = [0.0,1.0]
    for label in sorted(labels):
        print("Class %s precision = %s" % (label, metrics.precision(label)))
        print("Class %s recall = %s" % (label, metrics.recall(label)))
        print("Class %s F1 Measure = %s\n\n" % (label, metrics.fMeasure(label, beta=1.0)))
    metrics=BinaryClassificationMetrics(LR_P_O_test)
    print("Area under ROC =%s"% metrics.areaUnderROC)

## Calculating TF*IDF for Train Data...
## Creating Labeled points for Train Data...
## Calculating TF*IDF for Test Data...
## Creating Labeled points for Test Data...
Training Data 0.7561375
testing Data 0.766016713091922
[[ 129.   48.]
 [  36.  146.]]
Summary Stats
Precision = 0.766016713091922
Recall = 0.766016713091922
F1 Score = 0.766016713091922
Class 0.0 precision = 0.7818181818181819
Class 0.0 recall = 0.7288135593220338
Class 0.0 F1 Measure = 0.7543859649122807


Class 1.0 precision = 0.7525773195876289
Class 1.0 recall = 0.8021978021978022
Class 1.0 F1 Measure = 0.7765957446808511


Area under ROC =0.7655056807599181


### Decision Trees

Supplied TF IDf vector to decision trees and computed the metrics

In [ ]:
model = DecisionTree.trainClassifier(labeled_training_data, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=30, maxBins=100)
predictions = model.predict(labeled_training_data.map(lambda x: x.features))
labelsAndPredictions = labeled_training_data.map(lambda lp: lp.label).zip(predictions)
train_accuracy = labelsAndPredictions.filter(lambda v: v[0] == v[1]).count() / float(train_data.count())
print("Train accuracy is %s" % round(float(train_accuracy), 4))

# Evaluate model on test instances and compute test error
predictions = model.predict(labeled_testing_data.map(lambda x: x.features))
labelsAndPredictions = labeled_testing_data.map(lambda lp: lp.label).zip(predictions)
test_accuracy = labelsAndPredictions.filter(lambda v: v[0] == v[1]).count() / float(test_data.count())
print("Test accuracy is %s" % round(float(test_accuracy), 4))
metrics = MulticlassMetrics(labelsAndPredictions)
    
    # Overall statistics
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()
confMatrix=metrics.confusionMatrix().toArray()
print(confMatrix)
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)
    # Statistics by class
labels = [0.0,1.0]
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = %s\n\n" % (label, metrics.fMeasure(label, beta=1.0)))
metrics=BinaryClassificationMetrics(labelsAndPredictions)
print("Area under ROC =%s"% metrics.areaUnderROC)

### 10 fold Cross Validation

Partitioned the training data into 10 splits and tested the classifier accuracy with the remaining single partition.
data=labeled_training_data--For Logistic
data=train_tf---for NB
data=labeled_training_data--For Decision trees

In [ ]:
data = labeled_training_data
data.cache()
training =[]
val = []
partitions = data.randomSplit([.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],seed = 5)
for k in range(0,10):
    if k is 0:
        training.append(sc.union(partitions[1:10]))
    elif k is 9:
        training.append(sc.union(partitions[0:9]))
    else:
        p1 = sc.union(partitions[0:k])
        p2 = sc.union(partitions[k+1:10])    
        training.append(p1+p2)
    val.append(partitions[k])
    print(training[k].count(),val[k].count())

In [ ]:
accuracyLG = []
for k in range(0,10):    
    model = LogisticRegressionWithSGD.train(training[k])
    # Make prediction and test accuracy.
    predictionAndLabel = val[k].map(lambda p: (model.predict(p.features), p.label))
    #print(predictionAndLabel.take(3))
    accuracyLG.append(1.0 * predictionAndLabel.filter(lambda xv: xv[1] == xv[0]).count() / val[k].count())
    #print(accuracyLG[k])

avg_accuracyLG = sum(accuracyLG)/len(accuracyLG)
print(avg_accuracyLG)

In [ ]:
accuracyBNB = []
# Train a naive Bayes model.
for k in range(0,10):
    model= NaiveBayes.train(training[k], 1.0)
    # Make prediction and test accuracy.
    predictionAndLabel = val[k].map(lambda p: (model.predict(p.features), p.label))
    #print(predictionAndLabel.take(3))
    accuracyBNB.append(1.0 * predictionAndLabel.filter(lambda xv: xv[1] == xv[0]).count() / val[k].count())
    #print(accuracyBNB[k])

avg_accuracyBNB = sum(accuracyBNB)/len(accuracyBNB)
print("Avg accuracy for Naive Bayes:%s"% avg_accuracyBNB)

In [ ]:
accuracy = []
#model = []
for k in range(0,10):    
    model = DecisionTree.trainClassifier(training[k], numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=30, maxBins=100)
    predictions = model.predict(val[k].map(lambda x: x.features))
    labelsAndPredictions = val[k].map(lambda lp: lp.label).zip(predictions)
    accuracy.append(labelsAndPredictions.filter(lambda v: v[0] == v[1]).count() / float(val[k].count()))
    #print(accuracy[k])

avg_accuracy = sum(accuracy)/len(accuracy)
print("Avg accuracy for Decision Trees:%s" % avg_accuracy)

### Accuracy plot for three classifiers 

In [ ]:
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
print ('Success')
N = 3
TrainingAcc = (80.56, 75.72, 70.99)

ind = np.arange(N)  # the x locations for the groups
width = 0.25       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, TrainingAcc, width, color='r')
ax.set_ylim([50,100])
KfoldAcc = (74.5, 74.38, 67.22)
rects2 = ax.bar(ind + width, KfoldAcc, width, color='y')

TestAcc = (81.05, 76.88, 70.47)
rects3 = ax.bar(ind + 2*width, TestAcc, width, color='b')


# add some text for labels, title and axes ticks
ax.set_ylabel('Accuracy in %')
ax.set_title('Accuracies of NB , LOG and DT Classifiers')
ax.set_xticks(ind + width)
ax.set_xticklabels(('NB', 'LOG', 'DT'))

ax.legend((rects1[0], rects2[0], rects3[0]), ('Training', '10-FoldCV', 'Test'))


def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.02*height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)
plt.savefig('AccuracyPlot.png')

### Roc curve plot

In [ ]:
x=[1,0.74,0.50,0.271,0.135,0.039,0]
y=[1,0.98,0.93,0.80,0.63,0.38,0]
plt.plot(x,y,marker='o')
plt.plot([0,1],[0,1], ls='dotted')
plt.axis([0, 1, 0, 1])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.suptitle('ROC Curve', fontsize=14, fontweight='bold')
plt.savefig('ROC123.png')

### Getting correct and incorrect tweets

In [ ]:
print('## Building model on Train Data...')

labeled_training_data = train_data.map(lambda x: LabeledPoint(x[1],hashingTF.transform(x[0])))
labeled_training_data.persist()
print(labeled_training_data.take(2))
labeled_testing_data = test_data.map(lambda x: LabeledPoint(x[1],hashingTF.transform(x[0])))
labeled_testing_data.persist()

NBmodel = NaiveBayes.train(labeled_training_data,lambda_=1.0)

NB_P_O_train = labeled_training_data.map(lambda p: (NBmodel.predict(p.features), p.label))
accuracy = 1.0 * NB_P_O_train.filter(lambda x: x[0] == x[1]).count() / labeled_training_data.count()
print("Training Data %s" % accuracy)
predictions = NBmodel.predict(labeled_testing_data.map(lambda x: x.features))
NB_P_O_test = labeled_testing_data.map(lambda lp:  lp.label).zip(predictions)
#
accuracy = 1.0 * NB_P_O_test.filter(lambda x: x[0] == x[1]).count() / labeled_testing_data.count()
print("Testing Data %s " % accuracy)

NBmodel.save(sc, "NBModel")
NB_sameModel = NaiveBayesModel.load(sc, "NBModel")
j = predictions.coalesce(int(test_data.getNumPartitions()))
temp_testpred = test_data.zip(j)
# print(temp_testpred.take(2))
correctly_classified = temp_testpred.filter(lambda x: x[0][1] == x[1])
print(correctly_classified.take(7))